In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import numpy as np 
import openpyxl as xl
from openpyxl import load_workbook

k_fold = KFold(n_splits=10, shuffle=True, random_state=0)
sns.set()
wb=xl.Workbook()
sheet = wb.active
sheet.title='ML result'



train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')


train_test_data  = [train,test]

for dataset in  train_test_data:
    dataset['Title']=dataset['Name'].str.extract('([A-Za-z]+)\.',expand=False)


In [2]:
# for dataset  in train_test_data:
#     dataset['Title'] = dataset['Title'].replace(['Lady','Capt','Col','Countess','Don','Dona','Dr','Jonkheer','Major','Sir'],'Rare')
#     dataset['Title'] = dataset['Title'].replace(['Mlle','Ms'],'Miss')
#     dataset['Title'] = dataset['Title'].replace('Mme','Mrs')

# title_mapping = {'Mr':0,'Master':0.4,'Rev':0.8,'Miss':1.2,'Mrs':1.6,'Rare':2}

for dataset  in train_test_data:
    dataset['Title'] = dataset['Title'].replace(['Rev','Lady','Capt','Master','Col','Countess','Don','Dona','Dr','Jonkheer','Major','Sir'],'Rare')
    dataset['Title'] = dataset['Title'].replace(['Mlle','Ms'],'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme','Mrs')

title_mapping = {'Mr':0,'Miss':1,'Mrs':2,'Rare':3}

for dataset in train_test_data:
    dataset['Title'] = dataset['Title'].map(title_mapping)


In [3]:
pd.crosstab(train['Title'],train['Survived'])

Survived,0,1
Title,,
0,436,81
1,55,130
2,26,100
3,32,31


In [4]:
train.drop('Name',axis=1,inplace=True)
test.drop('Name',axis=1,inplace=True)
for dataset  in train_test_data:
    dataset['Sex'] = dataset['Sex'].replace('male',0)
    dataset['Sex'] = dataset['Sex'].replace('female',1)

In [5]:
train['Age'].fillna(train.groupby('Title')['Age'].transform('median'),inplace=True)
test['Age'].fillna(test.groupby('Title')['Age'].transform('median'),inplace=True)

for dataset in train_test_data:
    dataset.loc[ dataset['Age'] <= 18, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 18) & (dataset['Age'] <= 35), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 35) & (dataset['Age'] <= 45), 'Age'] = 2
    dataset.loc[ dataset['Age'] > 45, 'Age'] = 3


In [6]:
for dataset in train_test_data:
    dataset["Embarked"]=dataset['Embarked'].fillna('S')

embarked_mapping = {'S':0,'C':1,'Q':2}
for dataset in train_test_data:
    dataset['Embarked']=dataset['Embarked'].map(embarked_mapping)

In [7]:
train['Fare'].fillna(train.groupby('Pclass')['Fare'].transform('median'),inplace=True)
test['Fare'].fillna(test.groupby('Pclass')['Fare'].transform('median'),inplace=True)

for dataset in train_test_data:
    dataset.loc[ dataset['Fare'] <= 17, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 17) & (dataset['Fare'] <= 30), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 30) & (dataset['Fare'] <= 100), 'Fare'] = 2
    dataset.loc[ dataset['Fare'] > 100, 'Fare'] = 3

    

In [8]:
for dataset in train_test_data:
    dataset['Cabin'] =dataset['Cabin'].str[:1]
    
cabin_mapping = {'A':0,'B':0.4,'C':0.8,'D':1.2,'E':1.6,'F':2,'G':2.4,'T':2.8}

for dataset in train_test_data:
    dataset['Cabin']=dataset['Cabin'].map(cabin_mapping)

train['Cabin'].fillna(train.groupby('Pclass')['Cabin'].transform('median'),inplace=True)
test['Cabin'].fillna(test.groupby('Pclass')['Cabin'].transform('median'),inplace=True)


In [9]:

train['FamilySize'] = train['SibSp']+train['Parch'] + 1
test['FamilySize'] = test['SibSp']+test['Parch'] + 1

family_mapping={1:0, 2:0.2, 3:0.4, 4:0.6, 5:0.8, 6:1, 7:1.2, 8:1.4, 9:1.6, 10:1.8,11:2}
for dataset in train_test_data:
    dataset['FamilySize']=dataset['FamilySize'].map(family_mapping)


In [10]:

feature_drop = ['Ticket','SibSp','Parch']
train=train.drop(feature_drop,axis=1)
test=test.drop(feature_drop,axis=1)
train=train.drop(['PassengerId'],axis=1)

train_data = train.drop('Survived',axis=1)
target = train['Survived']


In [11]:
clf = KNeighborsClassifier(n_neighbors=13)
scoring = 'accuracy'
score = cross_val_score(clf, train_data,target,cv = k_fold,n_jobs=1,scoring=scoring)
kNN_Result = round(np.mean(score)*100,2)
print(score)
print('============================')
print('kNN Result : ' + str(kNN_Result))
print('============================')

[0.81111111 0.76404494 0.85393258 0.80898876 0.83146067 0.82022472
 0.84269663 0.78651685 0.82022472 0.84269663]
kNN Result : 81.82


81.82


In [12]:
clf = DecisionTreeClassifier()
scoring = 'accuracy'
score = cross_val_score(clf,train_data,target,cv=k_fold,n_jobs=1,scoring=scoring)
DecisionTree_Result = round(np.mean(score)*100,2)
print(score)
print('============================')
print('DecisionTree Result : ' + str(DecisionTree_Result))
print('============================')

[0.78888889 0.85393258 0.76404494 0.79775281 0.84269663 0.84269663
 0.82022472 0.79775281 0.74157303 0.82022472]
DecisionTree Result : 80.7


In [13]:
clf = RandomForestClassifier(n_estimators=13)
scoring = 'accuracy'
score = cross_val_score(clf,train_data,target,cv=k_fold,n_jobs=1,scoring=scoring)
RandomForest_Result = round(np.mean(score)*100,2)
print(score)
print('============================')
print('RandomForest Result : ' + str(RandomForest_Result))
print('============================')

[0.8        0.87640449 0.76404494 0.80898876 0.80898876 0.82022472
 0.82022472 0.79775281 0.7752809  0.85393258]
RandomForest Result : 81.26


In [14]:
clf = GaussianNB()
scoring = 'accuracy'
score = cross_val_score(clf,train_data,target,cv=k_fold,n_jobs=1,scoring=scoring)
GaussianNB_Result = round(np.mean(score)*100,2)
print(score)
print('============================')
print('GaussianNB Result : ' + str(GaussianNB_Result))
print('============================')

[0.85555556 0.73033708 0.74157303 0.74157303 0.70786517 0.80898876
 0.76404494 0.79775281 0.86516854 0.82022472]
GaussianNB Result : 78.33


In [15]:
clf = SVC()
scoring = 'accuracy'
score = cross_val_score(clf,train_data,target,cv=k_fold,n_jobs=1,scoring=scoring)
SVC_Result = round(np.mean(score)*100,2)
print(score)
print('============================')
print('SVC Result : ' + str(SVC_Result))
print('============================')

[0.83333333 0.79775281 0.83146067 0.80898876 0.85393258 0.80898876
 0.84269663 0.85393258 0.82022472 0.85393258]
SVC Result : 83.05


## Save Result Data in Result_save_file 

In [18]:
index=2
while True:
    if sheet['A{}'.format(index)].value == None:
        break
    index+=1
datas = (index-1,kNN_Result,DecisionTree_Result,RandomForest_Result,GaussianNB_Result,SVC_Result)
col_names = ['Index','kNN','DecisionTree','RandomForest','GaussianNB','SVM']

for seq, name in enumerate(col_names):
    sheet.cell(row=1, column=seq+1, value=name)
for seq, name in enumerate(datas):
    sheet.cell(row=index,column=seq+1,value=name)

print(sheet['A{}'.format(index)].value)

wb.save('testing.xlsx')

2


# **Make result File**

In [17]:
clf = SVC()
clf.fit(train_data,target)

test_data = test.drop('PassengerId',axis=1).copy()
prediction = clf.predict(test_data)
submission = pd.DataFrame({
    'PassengerId':test['PassengerId'],
    'Survived':prediction
})

submission.to_csv('submission_1.csv',index=False)

submission = pd.read_csv('submission_1.csv')
submission.head(30)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0
